[Click here for live site!](https://m-sender.github.io/notupyet)
<center><h1>Music Trends During the Pandemic</h1></center>
<center><b>Sam Broth and Max Sender</b></center>
<center>-------------------------------------------------------</center>

<center>1. Introduction</center>
<center>2. Data: Extraction, Transform and Load</center>
<center>3. Exploratory Data Analysis</center>
<center>4. Visualization</center>
<center>5. What it all means?</center>

<center><h1>1. Introduction</h1></center>

# Goals

# Our Data

# Questions and Hypothesis

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats
from scipy.interpolate import make_interp_spline
from functools import partial
import requests
import json
from lyricsgenius import Genius
import os
from dotenv import load_dotenv
load_dotenv()
import sqlite3
import nltk

<center><h1>2. Extraction, Transform and Load </h1></center>

We can do like 2.a, 2.b, etc for tidying and organizing and merging and such with explanations.

In [ ]:

tracks_df = pd.read_csv("../datasets/tracks.csv")
artist_df = pd.read_csv("../datasets/artists.csv")
final_df = pd.read_csv("../datasets/Final database.csv")
full_df = pd.read_csv("../datasets/Database to calculate popularity.csv")



# Explain Data sets

# Tidying up our data

In [ ]:
#tracks_df.sort_values('popularity', ascending=False)[tracks_df['popularity']>50].head()
tracks_df
#find the most popular tracks
tracks_df.sort_values('popularity', ascending=False).head(10)

In [ ]:
mental_health_df = pd.read_csv("../datasets/mentalHealth_searchTrend.csv")


In [ ]:
mental_health_df

In [ ]:
full_df['date'] = pd.to_datetime(full_df['date'],dayfirst=True)
full_df


In [ ]:
full_df.iloc[305933].uri
full_df['id'] = [x.split('/')[-1] if type(x)==str else float('nan') for x in full_df.uri.values]

In [ ]:
f = partial(pd.to_datetime,yearfirst=True)
working_df = full_df[(full_df['date']>=f('01/01/2020')) & (full_df['date']<=f('06/30/2020'))].sort_values('date',ascending=False)

In [ ]:
working_df

In [ ]:
tracks_df[(tracks_df['name']=='Tongue Tied') & (tracks_df['artists']=="['Grouplove']")]

In [ ]:
holder_df = working_df.merge(tracks_df,on='id',how='inner')
holder_df.head()

<center><h1>3. Exploratory Data Analysis</h1></center>


We break down our data analysis into 3 parts:
* (3.1) Analysis and Representation of Provided Metrics
* (3.2) NLP sentiment analysis of the lyrics per genre
* (3.3) Meshing the data together and pursuing further analysis

<center><h3>3.1: Analysis and Representation of Provided Metrics</h3></center>

In [ ]:
#Code for plotting graphs and metrics
def createMetricComparison(metricName): #change to zscore // streams per week (look)
    fig, ax = plt.subplots(1,2,figsize=(25,12), sharey=True, sharex=False)
    title = metricName + ' Before and After Pandemic Starts'
    fig.suptitle(title,fontsize=16)

    

    #holder_df[holder_df['country']== 'USA'].plot.scatter(x='date',y='danceability', c='liveness',ax=ax)
    #group by dates, get scores for each in terms of danceability and apply weight, then sum, them plot
    #convert zscore = 
    holder_df[metricName] = (holder_df[metricName] - holder_df[metricName].mean())/holder_df[metricName].std()
    pre_pandemic = holder_df[(holder_df['date']<=f('03/10/2020'))] #  & (holder_df['date'].dt.day % 7 == 0)
    pandemic = holder_df[(holder_df['date']>f('03/10/2020'))] # & (holder_df['date'].dt.day % 7 == 0)
    pre_pandemic = pre_pandemic[pre_pandemic['country']=='USA']
    pandemic = pandemic[pandemic['country']=='USA']
  
    
    
    pp_groupby = pre_pandemic.groupby('date')[metricName].mean() 
    dp_groupby = pandemic.groupby('date')[metricName].mean() 
    #get zscore
    '''pp_groupby_z = (pp_groupby - pp_groupby.mean())/pp_groupby.std()
    dp_groupby_z = (dp_groupby - dp_groupby.mean())/dp_groupby.std()'''
    std_pre = pp_groupby.std()
    pp_groupby.plot(ax=ax[0],title='Before Pandemic',ylabel=metricName, xlabel='Date')
    dp_groupby.plot(ax=ax[1],title='After Pandemic Begins', xlabel='Date',ylabel=metricName)
    #plt.axhline(y=pp_groupby.mean(), color='r', linestyle='-')
    ax[0].hlines(y=pp_groupby.mean(), xmin="01-01-2020", xmax="03-10-2020", linewidth=2, color='r')
    ax[0].hlines(y=pp_groupby.mean()+std_pre, xmin="01-01-2020", xmax="03-10-2020", linewidth=2, color='y',linestyle='--')
    ax[0].hlines(y=pp_groupby.mean()-std_pre, xmin="01-01-2020", xmax="03-10-2020", linewidth=2, color='y',linestyle='--')
    
    #plt.axhline( y=dp_groupby.mean(), color='r', linestyle='-')
    ax[1].hlines(y=pp_groupby.mean()+std_pre, xmin="03-11-2020", xmax="06-30-2020", linewidth=2, color='y',linestyle='--')
    ax[1].hlines(y=pp_groupby.mean()-std_pre, xmin="03-11-2020", xmax="06-30-2020", linewidth=2, color='y',linestyle='--')
    
    ax[1].hlines(y=dp_groupby.mean(), xmin="03-11-2020", xmax="06-30-2020", linewidth=2, color='r')

In [ ]:
createMetricComparison('danceability')

In [ ]:
metric_list = ['danceability','energy','loudness','speechiness','acousticness','instrumentalness','liveness','valence','tempo','duration_ms','time_signature','key','mode']
for metric in metric_list:
    createMetricComparison(metric)

<center><h3>3.2: NLP sentiment analysis of the lyrics per genre</h3></center>

Using genius API we obtained lyrics and stored in sqlite file for use, (see lyricgrabber)

In [ ]:
conn = sqlite3.connect('lyric_store.db')

lyric_df = pd.read_sql_query("SELECT * FROM lyrics", conn)
nltk.download('stopwords')

In [ ]:
lyric_df.rename(columns={'song':'title'},inplace=True)

In [ ]:
pre_pandemic_lyr = working_df[(working_df['date']<=f('03/10/2020')) & (holder_df['date'].dt.day % 7 == 0)]
pandemic_lyr = working_df[(working_df['date']>f('03/10/2020')) & (holder_df['date'].dt.day % 7 == 0)]
full_lyr = pd.merge(pre_pandemic_lyr,pandemic_lyr,how='outer')
full_lyr = pd.merge(full_lyr,lyric_df,how='outer')

In [ ]:
display(full_lyr.head())
display(lyric_df.head())

In [ ]:
#Using the lyric_df, we can generate a list of words and their frequencies for each song.
#We should make new df with song and then list as well as each weeks collective for analysis
lyric_df['words'] = None
word_stops = nltk.corpus.stopwords.words('english')

for index, row in lyric_df.iterrows():
        song_word_hold = {}
        words = row['lyric'].split()
        for word in words:
            if word.lower() in song_word_hold:
                song_word_hold[word.lower()] += 1
            else:
                song_word_hold[word.lower()] = 1
        lyric_df.loc[index]['words'] = list(song_word_hold.items())
lyric_df.head()
#for each song, get the lyrics, split into words, and count the frequency of each word.
#each song from the top 100 will be analyzed per week


In [ ]:
#Code for per week/day analysis
lyric_df.groupby('date').lyric.apply(' '.join).reset_index()
#for each song, get the lyrics, split into words, and count the frequency of each word.
#each song from the top 100 will be analyzed per week


<center><h3>3.3: Meshing the data together and pursuing further analysis</h3></center>

<center><h1>4. Visualization</h1></center>

<center><h1>5. What it all means?</h1></center>

Conclusion